## Use Lora for Adaption

### A Quick Overview

#### LoRa(ICLR2021)
<img width="380" height="380" src="../figs/lora/lora.png">

Low-Rank Adaptation, or [LoRA](https://github.com/microsoft/LoRA), freezes the pretrained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture, greatly reducing the number of trainable parameters for downstream tasks. Compared to GPT-3 175B fine-tuned with Adam, LoRA can reduce the number of trainable parameters by 10,000 times and the GPU memory requirement by 3 times.

#### AdaLoRa(ICLR2023)
[AdaLoRA](https://github.com/QingruZhang/AdaLoRA) adaptively allocates the parameter budget among weight matrices according to their importance score. In particular, AdaLoRA parameterizes the incremental updates in the form of singular value decomposition. Such a novel approach allows for the effective pruning of the singular values of unimportant updates, which is essential to reduce the parameter budget but circumvent intensive exact SVD computations.

### Application in our framework
For each AttentionBlock in ImageEncoder, we replace the two Linear Layers in Attention and Mlp with LoRa Linear Layer and AdaLoRa Layer, i.e., one example [here](../models/ImageEncoder/vit/lora_block.py)

### Training
In SAM, EfficientSAM, and MobileSAM, the adjustment of models using Lora is supported. The -mod option can be used to specify the fine-tuning method:
``python train.py -net mobile_sam -dataset REFUGE -data_path data/REFUGE -sam_ckpt checkpoint/mobile_sam/mobile_sam.pt -image_size 256 -vis 100 -exp_name tiny-mobile-isic-256 -encoder tiny_vit -mod sam_lora``



### Performance VS Adapter
#### REFUGE
| Baseline     | Backbone  | mode | DICE   | mIou | Memory  |
| ------------ | --------- | ------ | ---- | ------- | ------------ |
| EfficientSAM | VIT-Small | Adapter | 0.8691 | 0.7915 | 21275 M  |
| EfficientSAM | VIT-Small | Lora | 0.8573 | 0.7703 | 22777 M |
| EfficientSAM | VIT-Small | AdaLora | 0.8558 | 0.7596 | 22779 M |
| MobileSAM | Tiny-Vit | Adapter | 0.9330 | 0.8812 | 10255M |
| MobileSAM | Tiny-Vit | Lora | 0.9107 | 0.8436 | 10401M |
| MobileSAM | Tiny-Vit | AdaLora | 0.8863 | 0.8031 | 10401M |

### Curve of loss and performance
**based on MobileSAM(Tiny-Vit) model and REFUGE dataset**

#### Adapter
<p float="left">
  <img src="../figs/lora/MobileSAM-Ti (Adapter)_loss.png" width="400" />
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  <img src="../figs/lora/MobileSAM-Ti (Adapter)_performance.png" width="400" /> 
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</p>

#### LoRa
<p float="left">
  <img src="../figs/lora/MobileSAM-Ti (Lora)_loss.png" width="400" />
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  <img src="../figs/lora/MobileSAM-Ti (Lora)_performance.png" width="400" /> 
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</p>

#### AdaLoRa
<p float="left">
  <img src="../figs/lora/MobileSAM-Ti (AdaLora)_loss.png" width="400" />
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  <img src="../figs/lora/MobileSAM-Ti (AdaLora)_performance.png" width="400" /> 
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</p>

It can be seen that the training method using Adapter is more stable.